# Подключение библиотек

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.math import confusion_matrix

# Описание функций для получения ключевых точек

In [ ]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# функция определения положения руки и получения ключевых точек
def hand_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #перевод изображения в формат RGB
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) #перевод изображения в формат BGR
    return image, results

# функция отрисовки ключевых точек на изображении
def draw_landmarks(image, results):
    if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS,
                                         mp_drawing.DrawingSpec(color=(0, 0, 255),
                                                               thickness=2, circle_radius=4),
                                         mp_drawing.DrawingSpec(color=(0, 255, 0),
                                                               thickness=2, circle_radius=2))

# функция извлечения ключевых точек               
def extract_keypoints(results):
    hand = np.array([[res.x, res.y, res.z] for res in results.multi_hand_landmarks[0].landmark]).flatten() if results.multi_hand_landmarks else np.zeros(63)
    return hand

# Сбор жестовых данных

In [ ]:
DATA_PATH = os.path.join('DATA_3') # корневая папка, в которой будут храниться данные
actions = np.array(['move_left', 'move_straight', 'stop', 'move_right'])  # название жестов
number_sequences = 30 # количество записываемых видео для каждого жеста
sequence_length = 30 # количество кадров

# создание папок для каждого видео
for action in actions:
    for sequence in range(0,number_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action,
                                    str(sequence)))
        except:
            pass
        
cap = cv2.VideoCapture(0) # подключение к камере
with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    
    for action in actions:
        for sequence in range(0,number_sequences):
            for frame_num in range(sequence_length):
                
                ret, frame = cap.read() # считывание кадра
                
                image, results = hand_detection(frame, hands)
                
                draw_landmarks(image, results)
                
                if frame_num == 0:
                    
                    cv2.putText(image, 'Start',
                               (120, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                    
                    cv2.putText(image, 'gesture {} video number {}'.format(action, sequence),
                               (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    
                    cv2.imshow('Image', image)
                    cv2.waitKey(1000)
                    
                else:
                    
                    cv2.putText(image, 'gesture {} video number {}'.format(action, sequence),
                               (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    
                    cv2.imshow('Image', image)
                
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    
    cap.release()
    cv2.destroyAllWindows()

# Считывание записанных данных

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}

sequences, labels = [], []
for action in actions:
    for sequence in range(number_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)), allow_pickle=True)
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
        
X = np.array(sequences)
y = to_categorical(labels).astype(int)

# Обучение нейронной сети

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

model = Sequential()
model.add(LSTM(256, return_sequences=True, activation='relu', input_shape=(30,63)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# Вывод графиков

In [ ]:
history = model.fit(X_train, y_train, epochs=100, verbose=1, validation_data=(X_test, y_test))
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('точность модели')
plt.ylabel('точность')
plt.xlabel('эпоха')
plt.legend(['обучение', 'тест'], loc='upper left')
plt.show()

# Распознавание жестов в реальном времени

In [ ]:
sequence = []
sentence = ''
threshold = 0.95

cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        
        ret, frame = cap.read()
        
        image, results = hand_detection(frame, hands)
        
        draw_landmarks(image, results)
        
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            
            if res[np.argmax(res)] >= threshold:
                if len(sentence) > 0:
                    if actions[np.argmax(res)] != sentence:
                        sentence = actions[np.argmax(res)]
                else:
                    sentence = actions[np.argmax(res)]
        
        cv2.rectangle(image, (0,0), (400,40), (245,117,16), -1)
        cv2.putText(image, sentence, (3,30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),
                   2, cv2.LINE_AA)
        cv2.putText(image, str(float('{:.3f}'.format(res[np.argmax(res)]))), (330,30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),
                   2, cv2.LINE_AA)
        cv2.imshow('gesture recognition', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()